# Merge CORINE and LISA land cover

This notebook merges <a href="http://www.umweltbundesamt.at/umweltsituation/raumordnung/rp_projekte/rp_corine/">CORINE</a> and <a href="http://www.landinformationsystem.at/de-at/kartierungsdienste/landbedeckung.aspx">LISA</a> land cover datasets and converts CORINE land cover types into LISA land cover types based on a specified conversion map.

This notebook uses GRASS GIS (7.0.4), and must be run inside of a GRASS environment (start the jupyter notebook server from the GRASS command line).
<a href="https://grass.osgeo.org">GRASS GIS</a>

#### Required packages

<a href="https://github.com/numpy/numpy">numpy</a> <br />
<a href="https://github.com/pydata/pandas">pandas</a> <br />

#### Variable declarations

landcover_corine — name of CORINE land cover vector map <br />
landcover_lisa — name of LISA land cover vector map <br />
landcover — name of the merged land cover dataset <br />
conversion_map_filename — path to csv file mapping land cover types between CORINE and LISA datasets

In [1]:
landcover_corine = 'landcover_corine'

In [2]:
corine_type_field = 'value'

In [3]:
landcover_lisa = 'landcover_lisa'

In [4]:
lisa_type_field = 'lbtyp'

In [5]:
landcover = 'landcover'

In [75]:
conversion_map_filename = ""

#### Import statements

In [67]:
import pandas
import numpy as np

from grass import script as gscript
from grass.pygrass.vector import VectorTopo
from grass.pygrass.vector.table import DBlinks

#### Function declarations

connect to attribute table

In [8]:
def connectToAttributeTable(map):
    vector = VectorTopo(map)
    vector.open(mode='r')
    dblinks = DBlinks(vector.c_mapinfo)
    link = dblinks[0]
    return link.table()

overlay a vector map on an underlying vector map using a specified operator ('and', 'or', 'not', etc.)

In [9]:
def overlay(overlay, underlay, operator, output):
    gscript.read_command('v.overlay',
                         ainput=overlay,
                         binput=underlay,
                         operator=operator,
                         output=output,
                         overwrite=True)

#### Utility functions

export vector map to a shapefile

In [73]:
def exportVectorToShapefile(map, output):
    gscript.read_command('v.out.ogr',
                         input=map,
                         format='ESRI_Shapefile',
                         output=output,
                         flags='e',
                         overwrite=True)
    # flags='e' — use ESRI-style .prj file format

#### Merge land cover datasets

cut out the extent of LISA land cover dataset from the CORINE dataset

In [22]:
overlay(overlay=landcover_corine, underlay=landcover_lisa, operator='not', output=landcover + '_extended')

combine the cut CORINE dataset with the LISA dataset

In [27]:
overlay(overlay=landcover_lisa, underlay=(landcover + '_extended'), operator='or', output=landcover)

#### Convert land cover types

get attribute table

In [14]:
table = connectToAttributeTable(map=landcover)
table.filters.select()
columns = table.columns
cursor = table.execute()
result = np.array(cursor.fetchall())
cursor.close()

combined_table = pandas.DataFrame(result, columns=columns).set_index('cat')

load convsersion map

In [12]:
conversion_map = pandas.read_csv(conversion_map_filename).set_index('CORINE')

In [13]:
conversion_map

,LISA
CORINE,
1110,2
1120,2
1210,2
1220,2
1240,2
1420,2
2110,12
2210,12
2311,12


In [15]:
combined_lisa_type_field = 'a_' + lisa_type_field
combined_corine_type_field = 'b_a_' + corine_type_field

remapped = []
for index, row in combined_table[[combined_lisa_type_field, combined_corine_type_field]].iterrows():
    if row[combined_lisa_type_field] is not None:
        value = row[combined_lisa_type_field]
    else:
        value = conversion_map.get_value(row[combined_corine_type_field], 'LISA')
    remapped.append(value)

In [16]:
remapped_table = pandas.DataFrame({'cat':combined_table.index,
                                   'type':remapped}).set_index('cat')

replace attribute table

In [68]:
table.create([(u'cat', u'integer'), (u'type', u'integer')], overwrite=True)

In [69]:
table.insert([ (index, row['type']) for index, row in remapped_table.iterrows() ], many=True)

In [70]:
table.conn.commit()

#### Export shapefile

In [74]:
exportVectorToShapefile(landcover, "")